In [ ]:
import chipwhisperer as cw
import trsfile as trs

# Function:
# get the traces and plaintext list from a given file and convert into two arrays
# Arguments:
# - file_path: the file path of the trace file 
# - file_type: the file type of the trace file, can either be a CW file or a .trs file
# - time_start: the start sample of each trace, will default to 0
# - time_end: the end sample of each trace, will default to the length of the trace
def get_traces_from_file(file_path, file_type, time_start=None, time_end=None) :
    
    print("Attempting to open " + file_path + " as a " + file_type + " file...")

    # ChipWhisperer File Type
    if file_type == "CW" :
        try:
            project = cw.open_project(file_path)
        except OSError:
            print("Unable to find ChipWhisperer file")
            return None, None
            
        print("Total Number of Traces: " + project.traces.seg_len)
        
        trace_list=[]
        plaintext_list=[]
        
        for trace in project.traces:
            trace_list.append(trace.wave[time_start:time_end]) # TODO: This implementation needs to be tested
            plaintext_list.append(trace.textin)
        
        return trace_list, plaintext_list 
        
    # Trs File Tpe    
    elif file_type == ".trs" :

        with trs.open(file_path, 'r') as traces : 
            for header, value in traces.get_headers().items():
                print(header, '=', value)
            print("===========================================")    
            print("Total Number of Traces: " + str(len(traces)))
            
            trace_list=[]
            plaintext_list=[]
            
            for i, trace in enumerate(traces[0:len(traces)]): # enumerate over all traces in file
                # if no time start/end put entire trace into array
                if time_start is None :
                    time_start = 0
                if time_end is None :
                    time_end = len(trace)
                trace_list.append(trace[time_start:time_end]) # read specific time samples
                plaintext_list.append(trace.parameters['INPUT'].value[:])
                                
            return trace_list, plaintext_list
    else :
        print("Unknown File Type")
        return None, None     

In [ ]:
# Test reading on trs file
traces_list, plaintext = get_traces_from_file("traces1.trs", ".trs", 0, 255)